# Matched-filters

This notebook will provide a look at using EQcorrscan's Tribe objects for matched-filter detection of earthquakes.

This notebook extends on the ideas covered in the [Quick Start](quick_start.ipynb) notebook. In particular this
notebook also covers:
1. Concurrent execution of detection workflows for more efficient compute utilisation with large datasets;
2. Use of local waveform databases using [obsplus](https://github.com/niosh-mining/obsplus);
3. Cross-correlation pick-correction using the `lag_calc` method.

## Set up

We are going to focus in this notebook on using local data. For examples of how to directly use data from online providers
see the [Quick Start](quick_start.ipynb) notebook. 

To start off we will download some data - in your case this is likely data that you have either downloaded from one or more
online providers, or data that you have collected yourself. At the moment we don't care how those data are organised, as long
as you have continuous data on disk somewhere. We will use [obsplus](https://github.com/niosh-mining/obsplus) to work out
what data are were and provide us with the data that we need when we need it.

Obsplus is great and has more functionality than we expose here - if you make use of obsplus, please cite the 
paper by [Chambers et al., (2021)](https://joss.theoj.org/papers/10.21105/joss.02696).

As in the [Quick Start](quick_start.ipynb) example, we will control the output level from EQcorrscan using logging.

In [2]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s\t%(name)s\t%(levelname)s\t%(message)s")

Logger = logging.getLogger("TutorialLogger")

We will use the March 2023 Kawarau swarm as our case-study for this. This was an energetic swarm that
was reported by New Zealand's GeoNet monitoring agency and discussed in a news article [here](https://www.geonet.org.nz/response/VJW80CGEPtq0JPCBHlNaR).

We will use data from ten stations over a duration of two days. The swarm lasted longer than this, but
we need to limit compute resources for this tutorial! Feel free to change the end-date below to run
for longer.  To be kind to GeoNet and not repeatedly get data from their FDSN service we are going to
grab a folder of data I downloaded earlier.

In [6]:
%matplotlib inline

from obspy import UTCDateTime

starttime, endtime = UTCDateTime(2023, 3, 17), UTCDateTime(2023, 3, 19)
stations = ['EDRZ', 'KAFS', 'LIRZ', 'MARZ', 'MKRZ', 'MWFS', 'OMRZ', 'OPRZ', 'TARZ', 'WKHS']

# Uncomment this if you change starttime, endtime or stations to get the correct data.
# import os
# from obspy.clients.fdsn import Client
# 
# if not os.path.isdir("waveforms"):
#     os.makedirs("waveforms")
# client = Client("GEONET")
# date, chunk_length = starttime, 3600
# while date < endtime:
#     for station in stations:
#         for instrument in ["E", "H"]:
#             for component in ["Z", "N", "E", "1", "2"]:
#                 channel = f"{instrument}H{component}"
#                 try:
#                     st = client.get_waveforms(
#                         starttime=date,
#                         endtime=date + chunk_length,
#                         station=station,
#                         channel=channel,
#                         network="NZ", location="*")
#                 except Exception as e:
#                     Logger.debug(f"Could not download {station}.{channel} between "
#                                  f"{date} and {date + chunk_length} due to {e}")
#                     continue
#                 st.write(f"waveforms/{station}.{channel}_{date.strftime('%Y%m%dT%H%M%S')}.ms", 
#                          format="MSEED")
#     date += chunk_length

# Run this if you are just running the example:
import urllib.request

urllib.request.urlretrieve(
    "https://www.dropbox.com/scl/fo/95l35cq4ucb9yb43ehtng/h?rlkey=mxoka7gakouebh287pqz9nqza&dl=0",
    "./waveforms")

('./waveforms', <http.client.HTTPMessage at 0x7fe2c8b28910>)

In [7]:
ls

clustering.rst  matched-filter.ipynb  quick_start.ipynb  template-creation.rst
mag-calc.rst    processing_flow.png   subspace.rst       waveforms
